In [1]:
import dgl, os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

In [9]:
movie_df = pd.read_csv('sample_files/movies_2.csv')

In [10]:
users_df = pd.read_csv('sample_files/users_2.csv')

In [11]:
ratings_df = pd.read_csv('sample_files/ratings_2.csv')

In [39]:
ratings_df

,user_id,movie_id,ratings
0,0,0,1
1,0,1,5
2,0,2,5
3,1,1,5
4,1,3,5


In [15]:
graph_data = {
        ('user','rates','movie') : (ratings_df['user_id'].to_numpy(), ratings_df['movie_id'].to_numpy()),
        ('movie','rev-rates','user') : (ratings_df['movie_id'].to_numpy(), ratings_df['user_id'].to_numpy())
    }

In [25]:
movie_hetero_graph = dgl.heterograph(graph_data)

In [28]:
movie_hetero_graph

Graph(num_nodes={'movie': 4, 'user': 2},
      num_edges={('movie', 'rev-rates', 'user'): 5, ('user', 'rates', 'movie'): 5},
      metagraph=[('movie', 'user', 'rev-rates'), ('user', 'movie', 'rates')])

In [33]:
movie_hetero_graph.nodes['user'].data['features'] = torch.ones(movie_hetero_graph.num_nodes('user'),1)
movie_hetero_graph.nodes['movie'].data['features'] = torch.ones(movie_hetero_graph.num_nodes('movie'),1)

In [38]:
movie_hetero_graph.nodes['movie'].data['features'] = torch.ones(movie_hetero_graph.num_nodes('movie'),1)

10

In [54]:
movie_hetero_graph.edges['rates'].data['features'] = torch.ones(movie_hetero_graph.number_of_edges()//2,1)
movie_hetero_graph.edges['rev-rates'].data['features'] = torch.ones(movie_hetero_graph.number_of_edges()//2,1)

In [55]:
train_g = movie_hetero_graph

In [91]:
dim_dict = {'user': train_g.nodes['user'].data['features'].shape[1],
            'movie': train_g.nodes['movie'].data['features'].shape[1],
            'edge_dim': train_g.edges['rates'].data['features'].shape[1],
            'hidden_dim' : 128,
            'out_dim': 4
           }

In [92]:
def construct_negative_graph(graph, k, etype_list):
    HM = {}
    for etype in etype_list:
        utype, _, vtype = etype
        src, dst = graph.edges(etype=etype)
        neg_src = src.repeat_interleave(k)
        neg_dst = torch.randint(0, graph.num_nodes(vtype), (len(src) * k,))
        HM[etype] = (neg_src[:], neg_dst[:])

    return dgl.heterograph(
        HM,
        num_nodes_dict={ntype: graph.num_nodes(ntype) for ntype in graph.ntypes})


In [93]:
train_neg = construct_negative_graph(movie_hetero_graph, 2, movie_hetero_graph.canonical_etypes)

In [94]:
import dgl.function as fn

class GNNLayer(nn.Module):

    def __init__(self, input_graph, dim_dict):
        
        super().__init__()
        self.node_in_feats = dim_dict['out_dim']
        self.node_out_feats = dim_dict['out_dim']
#         self.edge_fc = nn.ModuleDict({
#             name[1] : nn.Linear(dim_dict['edge_dim'], self.node_out_feats*self.node_out_feats) for name in input_graph.canonical_etypes
#         })
        
    ## added a differnet edge network : check if it makes any difference
        self.edge_fc = nn.ModuleDict({
            name[1] : nn.Sequential(
                nn.Linear(dim_dict['edge_dim'], dim_dict['hidden_dim']),
                nn.ReLU(),
                nn.Linear(dim_dict['hidden_dim'], self.node_out_feats * self.node_out_feats),
                nn.Dropout(p=0.5)
                # check if dropout is necessary
            ) for name in input_graph.canonical_etypes
        })
    
    def udf_u_mul_e(self, nodes):
#         print("Important Shapes :", nodes.src['node_weights'], nodes.data['edge_weights'].shape)
        return {'m': nodes.src['node_weights'] * nodes.data['edge_weights'] }
    
    def reduce_func(self, nodes):
        return {'h': torch.mean(nodes.mailbox['m'], dim=1)}
    
    def forward(self, pos_g, feat_dict, edge_dict):

        funcs = {}
        for c_etype in pos_g.canonical_etypes:
            
            srctype, etype, dsttype = c_etype
            
#             print("Source",srctype, etype, "Destination",dsttype)
            
            node_weights = feat_dict[srctype].unsqueeze(-1)
#             print("Source type", srctype)
            print("Node weights shape:", node_weights.shape)

#             pos_g.nodes[srctype].data['node_weights_%s' % etype] = node_weights
            pos_g.nodes[srctype].data['node_weights'] = node_weights

            edge_weights = self.edge_fc[etype](edge_dict[etype]).view(-1, self.node_in_feats, self.node_out_feats)
            
            print("Edge weights shape", edge_weights.shape)
#             print("Edge weights :",edge_weights.shape, edge_dict[etype])
            
            pos_g[etype].edata['edge_weights'] = edge_weights
#             pos_g[etype].edata['edge_weights_%s' % etype] = edge_weights
            
#             funcs[etype] = (fn.u_mul_e('node_weights_%s' % etype, 'edge_weights_%s' % etype,'m'), self.reduce_func)
            funcs[etype] = (self.udf_u_mul_e, self.reduce_func)

        pos_g.multi_update_all(funcs, 'mean')
        results = {ntype : pos_g.nodes[ntype].data['h'].sum(dim=1) for ntype in pos_g.ntypes}
        
#         print("Results : ",results)
        
        return results

class GNNMODEL(nn.Module):

    def __init__(self, G, dim_dict, num_step_message_passing):

        super(GNNMODEL, self).__init__()
        
        # Added an activation function here : check if actually required. 
        self.project_node_features = nn.ModuleDict({
            name[0] : nn.Sequential(nn.Linear(dim_dict[name[0]], dim_dict['out_dim']), nn.Softmax(dim=0)) for name in G.canonical_etypes
        })
        
        self.num_step_message_passing = num_step_message_passing
        self.gnn_layer = GNNLayer(G, dim_dict=dim_dict)
    
    def forward(self, pos_g, node_feats, edge_feats):
        
#         print("Pre-projected Node Features :", node_feats)
        
        for feat in node_feats:
            node_feats[feat] = self.project_node_features[feat](node_feats[feat])
            
#         print("Projected Node Features : ", node_feats)
        
        for i in range(self.num_step_message_passing):
            node_feats = self.gnn_layer(pos_g, node_feats, edge_feats)
#             print("Layer :"+str(i))
#             print(node_feats)
#         return self.predictor(g, node_feats), self.predictor(node_subgraph_negative, node_feats)
        
        for feat in node_feats:
            node_feats[feat] = F.relu(node_feats[feat])
    
        return node_feats


In [95]:
class CosinePrediction(nn.Module):
    """
    Scoring function that uses cosine similarity to compute similarity between user and item.

    Only used if fixed_params.pred == 'cos'.
    """
    def __init__(self):
        super().__init__()

    def forward(self, graph, h):
        with graph.local_scope():
            for etype in graph.canonical_etypes:
                try:
                    graph.nodes[etype[0]].data['norm_h'] = F.normalize(h[etype[0]], p=2, dim=-1)
                    graph.nodes[etype[2]].data['norm_h'] = F.normalize(h[etype[2]], p=2, dim=-1)
                    graph.apply_edges(fn.u_dot_v('norm_h', 'norm_h', 'cos'), etype=etype)
                except KeyError:
                    pass  # For etypes that are not in training eids, thus have no 'h'
            ratings = graph.edata['cos']
        return ratings

class GNNPredictor(nn.Module):
    
    def __init__(self, input_graph, dim_dict):
        
        super(GNNPredictor, self).__init__()
        
        self.gnn = GNNMODEL(G = input_graph,
                            dim_dict = dim_dict,
                               num_step_message_passing = 2)
        self.predictor = CosinePrediction()
    
    def forward(self, g, neg_g, node_feats, edge_feats):

        node_feats = self.gnn(g, node_feats, edge_feats)
#         print("Predicted Node Features",node_feats)
        
        return self.predictor(g, node_feats), self.predictor(neg_g, node_feats)
    
    def get_repr(self, g, node_feats, edge_feats):
        """Returns the embedded representation given block made from sampling neighboring nodes."""
        
        node_feats = self.gnn(g, node_feats, edge_feats)
        return node_feats

In [96]:
mpnn_model = GNNPredictor(train_g, dim_dict)
optimizer = torch.optim.Adam(mpnn_model.parameters(), lr=0.0001,weight_decay=0)

In [97]:
for e in range(1):
    
    optimizer.zero_grad()
    
    node_features = train_g.ndata['features']
    edge_features = train_g.edata['features']
    
    edge_features_HM = {}
    for key, value in edge_features.items():
        edge_features_HM[key[1]] = value.to(torch.float)
    
    print(node_features)
    print(edge_features_HM)
    
    pos_score, neg_score = mpnn_model(train_g, train_neg, node_features, edge_features_HM)

    print('------------------------------------------------------------------------')
    
    loss = max_margin_loss(pos_score, neg_score)
    
    print("Epoch : {}, Training loss : {}".format(e, loss.item()))

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

{'movie': tensor([[1.],
        [1.],
        [1.],
        [1.]]), 'user': tensor([[1.],
        [1.]])}
{'rev-rates': tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.]]), 'rates': tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.]])}
Node weights shape: torch.Size([4, 4, 1])
Edge weights shape torch.Size([5, 4, 4])
Node weights shape: torch.Size([2, 4, 1])
Edge weights shape torch.Size([5, 4, 4])
Node weights shape: torch.Size([4, 4, 1])
Edge weights shape torch.Size([5, 4, 4])
Node weights shape: torch.Size([2, 4, 1])
Edge weights shape torch.Size([5, 4, 4])
------------------------------------------------------------------------
Epoch : 0, Training loss : 0.49263066053390503


In [63]:
def max_margin_loss(pos_score, neg_score, delta=0.5):

    all_scores = torch.empty(0)
    
    for etype in pos_score.keys():
        neg_score_tensor = neg_score[etype]
        pos_score_tensor = pos_score[etype]

        neg_score_tensor = neg_score_tensor.reshape(pos_score_tensor.shape[0], -1)
        scores = (neg_score_tensor - pos_score_tensor + delta).clamp(min=0)

        relu = nn.ReLU()
        scores = relu(scores)
        all_scores = torch.cat((all_scores, scores), 0)

    return torch.mean(all_scores)